## Part 1: Toronto Data Scrapping

In [1]:
# installing dependencies
!pip install bs4
!pip install requests
!pip install html5lib

In [2]:
# Importing requried libraries
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [3]:
# requesting url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url)

# creating object of BeautifulSoup
soup = BeautifulSoup(r.content, 'html5lib')

try:
    table_content = []
    table = soup.find('table')
    for row in table.findAll('td'):
        data_dict = {}
        if row.span.text == 'Not assigned':
            pass
        else:
            data_dict['PostalCode'] = row.p.text[0:3]
            data_dict['Borough'] = (row.span.text).split('(')[0]
            data_dict['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
            table_content.append(data_dict)

except Exception as e:
    print(e)

# converting content to data frame
df_toronto = pd.DataFrame(table_content)
df_toronto['Borough'] = df_toronto['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
# Sorting df_toronto dataframe
df_toronto.sort_values(by=['PostalCode'], ascending = True, inplace = True)

df_toronto.reset_index(inplace = True)

df_toronto.drop(columns = 'index', inplace = True)

# data frame created: Web scrapping done
df_toronto

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [4]:
df_toronto.shape

(103, 3)

## Part 2: Geocoder Latitude and Longitude (Using CSV file) as geocoder did not work

In [6]:
lat_lon_coords = pd.read_csv('Geospatial_Coordinates.csv')
lat_lon_coords.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df_toronto = pd.concat([df_toronto, lat_lon_coords], axis = 1, join = 'inner')
df_toronto.drop(columns = 'Postal Code', inplace = True)
df_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


## Part 3: Clustering Data

In [12]:
# importing dependencies
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [13]:
!pip install folium
import folium # map rendering library

print('Folium installed')

Folium installed


In [14]:
# Checking got unique Boroughs

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


### Filtering Borough containing Toronto

In [15]:
df_toronto = df_toronto[df_toronto['Borough'].str.contains('Toronto')]
df_toronto = df_toronto.reset_index(drop = True)
df_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106
2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
4,M4M,East Toronto,Studio District,43.659526,-79.340923
5,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
6,M4P,Central Toronto,Davisville North,43.712751,-79.390197
7,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
8,M4S,Central Toronto,Davisville,43.704324,-79.388790
9,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


### Getting coordinates of Toronto, Canada

In [17]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="T_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


### Creating Map of Toronto City with neighborhood

In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Four square API credentials

In [21]:
@hidden_cell
# Credentials for Foursquare API

CLIENT_ID = 'XXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXX' # your Foursquare Secret
VERSION = '20210628' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VZUTPY2FOKIONCNBNMKIAMBON14B5VVEJPEHMO10343YTRWE
CLIENT_SECRET:25E23UHN0YD4F1GDHQWD0104WT4Z5R3D0TMDWTV23AOGITFL


### Exploring All neighbourhood in the data. Taking 1st value of neighbourhood if multiple values are there

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):

        # Taking 1st neighborhood if multiple values are present
        if ',' in name:
            name = name.split(', ')[0]
            print(name)
        else:
            print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
df_toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )


The Beaches
The Danforth  East
The Danforth West
India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park
Summerhill West
Rosedale
St. James Town
Church and Wellesley
Regent Park
Garden District
St. James Town
Berczy Park
Central Bay Street
Richmond
Harbourfront East
Toronto Dominion Centre
Commerce Court
Roselawn
Forest Hill North & West
The Annex
University of Toronto
Kensington Market
CN Tower
Enclave of M5E
First Canadian Place
Christie
Dufferin
Little Portugal
Brockton
High Park
Parkdale
Runnymede
Enclave of M4L


#### Checking size of resulting dataframe

In [41]:
print(df_toronto_venues.shape)
df_toronto_venues.head()

(1587, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth East,43.685347,-79.338106,The Path,43.683923,-79.335007,Park


#### Number of venues were returned for each neighborhood

In [42]:
df_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
Brockton,24,24,24,24,24,24
CN Tower,16,16,16,16,16,16
Central Bay Street,65,65,65,65,65,65
Christie,16,16,16,16,16,16
Church and Wellesley,79,79,79,79,79,79
Commerce Court,100,100,100,100,100,100
Davisville,37,37,37,37,37,37
Davisville North,7,7,7,7,7,7


#### Number of unique categories from all the returned venues

In [43]:
print('There are {} uniques categories.'.format(len(df_toronto_venues['Venue Category'].unique())))

There are 231 uniques categories.


### Analyzing each neighborhood

In [90]:
# one hot encoding
df_toronto_onehot = pd.get_dummies(df_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_toronto_onehot['Neighborhood'] = df_toronto_venues['Neighborhood'] 
# neighborhoof_col_loc = df_toronto_onehot.columns.get_loc('Neighborhood')

# # move neighborhood column to the first column
# fixed_columns = [df_toronto_onehot.columns[164]] + list(df_toronto_onehot.columns[:-1])
# df_toronto_onehot = df_toronto_onehot[fixed_columns]

df_toronto_onehot.head()

,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [91]:
df_toronto_onehot.shape

(1587, 231)

#### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [92]:
df_toronto_grouped = df_toronto_onehot.groupby('Neighborhood').mean().reset_index()
df_toronto_grouped

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000
1,Brockton,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,CN Tower,0.000000,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.015385,0.000000,0.000000,0.015385,0.015385
4,Christie,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.012658,0.0000,0.0000,0.000,0.000,0.000,0.012658,0.000000,0.00,...,0.012658,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.025316
6,Commerce Court,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.020000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
7,Davisville,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.027027,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Dufferin,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


#### Function to sort venues in descentding order and selecting top 10 common venues from each neighborhood

In [94]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [156]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted7 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted7['Neighborhood'] = df_toronto_grouped['Neighborhood']

for ind in np.arange(df_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted7.iloc[ind, 1:] = return_most_common_venues(df_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted7.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Beer Bar,Farmers Market,Restaurant,Seafood Restaurant,Gourmet Shop,Sporting Goods Shop
1,Brockton,Café,Bakery,Breakfast Spot,Coffee Shop,Intersection,Pet Store,Furniture / Home Store,Stadium,Restaurant,Climbing Gym
2,CN Tower,Airport Lounge,Airport Service,Airport Terminal,Boutique,Airport,Boat or Ferry,Sculpture Garden,Rental Car Location,Bar,Coffee Shop
3,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Japanese Restaurant,Burger Joint,Restaurant,Salad Place,Ramen Restaurant
4,Christie,Grocery Store,Café,Park,Baby Store,Italian Restaurant,Candy Store,Athletics & Sports,Coffee Shop,Nightclub,Restaurant


### Clustering Neighborhood

In [157]:
# set number of clusters
kclusters = 5

df_toronto_grouped_clustering = df_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(type(list(kmeans.labels_)))

<class 'list'>


In [158]:
# add clustering labels
neighborhoods_venues_sorted7.insert(0, 'Cluster Labels', list(kmeans.labels_))
neighborhoods_venues_sorted7.set_index('Neighborhood')
df_toronto_merged = df_toronto

# merge df_toronto_grouped with df_toronto to add latitude/longitude for each neighborhood
df_toronto_merged = df_toronto_merged.join(neighborhoods_venues_sorted6.set_index('Neighborhood'), on='Neighborhood', how = 'inner')



In [168]:
print(df_toronto_merged.shape)
print(len(kmeans.labels_))
df_toronto_merged.reset_index(inplace = True)
df_toronto_merged

(17, 17)
38


,level_0,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Trail,Pub,Adult Boutique,Modern European Restaurant,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School
1,1,1,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106,2,Park,Convenience Store,Molecular Gastronomy Restaurant,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station
2,2,4,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Brewery,Bakery,Gastropub,American Restaurant,Café,Middle Eastern Restaurant,Gay Bar,Bookstore,Pet Store
3,3,5,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Bus Line,Swim School,Modern European Restaurant,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store
4,4,6,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Department Store,Park,Pizza Place,Breakfast Spot,Food & Drink Shop,Sandwich Place,Hotel,Adult Boutique,Middle Eastern Restaurant,Mexican Restaurant
5,5,7,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Coffee Shop,Clothing Store,Yoga Studio,Bagel Shop,Cosmetics Shop,Diner,Chinese Restaurant,Rental Car Location,Restaurant,Café
6,6,8,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Pizza Place,Dessert Shop,Sandwich Place,Café,Coffee Shop,Italian Restaurant,Gym,Sushi Restaurant,Fried Chicken Joint,Gas Station
7,7,11,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Playground,Trail,Modern European Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School
8,8,13,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Mediterranean Restaurant,Men's Store,Fast Food Restaurant,Bubble Tea Shop
9,9,16,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Bakery,Italian Restaurant,Restaurant,Gastropub,Beer Bar,Cocktail Bar,American Restaurant,Japanese Restaurant


### Visualizing clusters

In [169]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_toronto_merged['Latitude'], df_toronto_merged['Longitude'], df_toronto_merged['Neighborhood'], df_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining Clusters

In [170]:
# Cluster 1
df_toronto_merged.loc[df_toronto_merged['Cluster Labels'] == 0, df_toronto_merged.columns[[1] + list(range(5, df_toronto_merged.shape[1]))]]

,index,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,5,43.728020,-79.388790,0,Park,Bus Line,Swim School,Modern European Restaurant,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store
7,11,43.679563,-79.377529,0,Park,Playground,Trail,Modern European Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School
13,24,43.696948,-79.411307,0,Park,Jewelry Store,Trail,Sushi Restaurant,Adult Boutique,Miscellaneous Shop,Liquor Store,Lounge,Malay Restaurant,Market


In [171]:
# Cluster 2
df_toronto_merged.loc[df_toronto_merged['Cluster Labels'] == 1, df_toronto_merged.columns[[1] + list(range(5, df_toronto_merged.shape[1]))]]

,index,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,43.676357,-79.293031,1,Health Food Store,Trail,Pub,Adult Boutique,Modern European Restaurant,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School
2,4,43.659526,-79.340923,1,Coffee Shop,Brewery,Bakery,Gastropub,American Restaurant,Café,Middle Eastern Restaurant,Gay Bar,Bookstore,Pet Store
4,6,43.712751,-79.390197,1,Department Store,Park,Pizza Place,Breakfast Spot,Food & Drink Shop,Sandwich Place,Hotel,Adult Boutique,Middle Eastern Restaurant,Mexican Restaurant
5,7,43.715383,-79.405678,1,Coffee Shop,Clothing Store,Yoga Studio,Bagel Shop,Cosmetics Shop,Diner,Chinese Restaurant,Rental Car Location,Restaurant,Café
6,8,43.704324,-79.388790,1,Pizza Place,Dessert Shop,Sandwich Place,Café,Coffee Shop,Italian Restaurant,Gym,Sushi Restaurant,Fried Chicken Joint,Gas Station
8,13,43.665860,-79.383160,1,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Mediterranean Restaurant,Men's Store,Fast Food Restaurant,Bubble Tea Shop
9,16,43.651494,-79.375418,1,Coffee Shop,Café,Bakery,Italian Restaurant,Restaurant,Gastropub,Beer Bar,Cocktail Bar,American Restaurant,Japanese Restaurant
10,17,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Beer Bar,Farmers Market,Restaurant,Seafood Restaurant,Gourmet Shop,Sporting Goods Shop
11,18,43.657952,-79.387383,1,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Japanese Restaurant,Burger Joint,Restaurant,Salad Place,Ramen Restaurant
14,29,43.646435,-79.374846,1,Coffee Shop,Italian Restaurant,Seafood Restaurant,Restaurant,Hotel,Cocktail Bar,Bakery,Café,Pub,Japanese Restaurant


In [172]:
# Cluster 3
df_toronto_merged.loc[df_toronto_merged['Cluster Labels'] == 2, df_toronto_merged.columns[[1] + list(range(5, df_toronto_merged.shape[1]))]]

,index,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,1,43.685347,-79.338106,2,Park,Convenience Store,Molecular Gastronomy Restaurant,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station


In [173]:
# Cluster 4
df_toronto_merged.loc[df_toronto_merged['Cluster Labels'] == 3, df_toronto_merged.columns[[1] + list(range(5, df_toronto_merged.shape[1]))]]

,index,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,23,43.711695,-79.416936,3,Home Service,Music Venue,Garden,Adult Boutique,Molecular Gastronomy Restaurant,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant
